In [2]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from stop_words import get_stop_words
import matplotlib.pyplot as plt

from collections import Counter


filename = "../training.1600000.processed.noemoticon.csv"

In [5]:
df = pd.read_csv(filename, encoding='latin-1', names=["target", "id", "date", "flag", "user", "text"])
df = df[["text", "target"]]

In [6]:
# Clean text
text = df["text"]
text = text.str.replace(r'http?\S+', ' ', regex=True)
#text = text.str.replace(r'[#@]\S+', ' ', regex=True)
text = text.str.replace(r'@\S+', ' ', regex=True)
text = text.str.replace(r'[^\w\s]', ' ', regex=True)
text = text.str.replace(r'\n', ' ', regex=True)
text = text.str.replace(r' +', ' ', regex=True)
text = text.str.lower()
# Add cleaned text to df
df = df.drop(columns = ["text"])
df = pd.concat([text, df], axis=1, sort=False)
df.head()

,text,target
0,awww that s a bummer you shoulda got david ca...,0
1,is upset that he can t update his facebook by ...,0
2,i dived many times for the ball managed to sa...,0
3,my whole body feels itchy and like its on fire,0
4,no it s not behaving at all i m mad why am i ...,0


In [7]:
from sklearn import preprocessing

target = df[["target"]].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(target)
normalized = pd.DataFrame(x_scaled)
df["target"] = normalized

In [8]:
def rmStopWords(tweet, stop_words):
    text = tweet.split()
    text = ' '.join(word for word in text if word not in stop_words)
    return text

stop_words = get_stop_words('english')
stop_words = [''.join(c for c in s if c not in string.punctuation) for s in stop_words]
stop_words = [t.encode('utf-8') for t in stop_words]

# Preprocess all tweet data
pro_tweets = []
for tweet in df['text']:
    pro_stopw = rmStopWords(tweet, stop_words)
    pro_tweets.append(pro_stopw)

df['text'] = pro_tweets

backup = df
df = df.sample(n=1600000)
df.head()

,text,target
916312,tweeterizing from a mobile,1.0
1427558,date night its a surprise,1.0
369132,i know right i got work at 7am,0.0
1065837,listening to dancing in the moonlight by toplo...,1.0
942614,whoever forms the government it will be status...,1.0


In [9]:
tags = df.target
texts = df.text
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from tensorflow.keras import metrics
print('import done')

import done


In [ ]:
texts

In [ ]:
num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)

In [ ]:
texts

In [ ]:
# try a simple model first

def get_simple_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(num_max,)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',metrics.binary_accuracy])
    print('compile done')
    return model

def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

simple_model = get_simple_model()
check_model(simple_model,mat_texts,tags)

In [ ]:
simple_model.save("simple_model")

In [ ]:
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)

In [ ]:
def get_cnn_model_v3():    # added filter
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256, #!!!!!!!!!!!!!!!!!!!
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

cnn_model_v3 = get_cnn_model_v3()
check_model(cnn_model_v3,cnn_texts_mat,tags)

In [ ]:
cnn_model_v3.save("cnn_model_v3")

In [10]:
tweetsdf = pd.read_csv(open('clean.csv','rU'), encoding='utf-8', engine='c').drop(columns=["Unnamed: 0"])
tweetsdf.head()

/home/orlando/anaconda3/envs/tfgpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


,text,timestamp,user.name,user.screen_name,user.followers_count,state,hashtags,mentions
0,all in collusion together nojustice trumppence,2016-08-12 10:04:02.194,Red Octopus,redoctapus,531,LA,"['NOJUSTICE', 'TrumpPence']","['BarackObama', 'FBI', 'LorettaLynch', 'realDo..."
1,cnn newday clear trump deliberately throwing ...,2016-08-12 10:04:30.035,Beverly Spence,bspence5,2652,MD,"['CNN', 'newday', 'Trump', 'ISIS']",[]
2,kid you know suing someone thats the most bea...,2016-08-12 10:04:48.229,Rafael Alejandro,GinebraFilms,159,NJ,[],"['funnyordie', 'realDonaldTrump']"
3,i totally concur this election is just cra cr...,2016-08-12 10:04:53.571,Kim Wasson,kimseacret3,244,MD,[],"['mike4193496', 'realDonaldTrump']"
4,you are the co founder of isis you crooked ev...,2016-08-12 10:04:48.695,tom b,VNDISABLEDVET,68,TX,[],['HillaryClinton']


In [13]:
tweets = tweetsdf.sample(3000)["text"].apply(str)
num_max = 1000
# preprocess
le = LabelEncoder()
tok = Tokenizer(num_words=num_max)

mat_tweets = tok.texts_to_matrix(tweets,mode='count')

print(mat_tweets[:5])
print(mat_tweets.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(3000, 1000)


In [17]:
from tensorflow.keras.models import load_model
simple_model = load_model('simple_model')
simple_pred = simple_model.predict(mat_tweets)

In [18]:
simple_pred

array([[0.5930077],
       [0.5930077],
       [0.5930077],
       ...,
       [0.5930077],
       [0.5930077],
       [0.5930077]], dtype=float32)

In [25]:
tweetsdf.sample(5)

,text,timestamp,user.name,user.screen_name,user.followers_count,state,hashtags,mentions
428622,seriously he will never admit he said it even if it s recorded trumpeteers will follow him over cliff regardless,2016-09-08 02:42:43.680,Joy Sayler,jsayler,1028,NV,[],[]
187220,stay woke,2016-08-24 00:38:41.203,Dilfof2016,TaylorBclark20,787,TX,[],[]
480631,or maybe it will be a powerpoint presentation or an excel data dump the possibilities are endless,2016-09-10 18:36:10.964,(((Kerri))),klberney,357,NY,[],"['TerryOswald1', 'AP', 'HillaryClinton']"
268796,have your followers turn off quality filter amp validate mute folder twitter using this to censor our followers from us,2016-08-29 00:43:01.078,❤️Boricua⛪️,NYRican4Trump,1531,GA,[],['realDonaldTrump']
93378,just like the polls my advisors are rigged too donaldtrump on the campaign shakeup,2016-08-17 22:38:34.615,Jer-Jer,DALLASNITES,8287,IL,['DonaldTrump'],[]


In [23]:
pd.set_option('display.max_colwidth', -1)

/home/orlando/anaconda3/envs/tfgpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.
